In [1]:
! python run_experiments.py 30 25 2 1 True

Starting job with args:
['run_experiments.py', '30', '25', '2', '1', 'True']
Loaded data shapes:
(10800, 80, 64) (10800, 1)
Using CPU
X shape: (10800, 80, 64, 1)
y shape: (10800, 2)
New model name: ./model/EEGNet_fusion_Executed_103sub_0.h5
Model EEGNet_fusion_Executed_103sub total training time was 1690.6588184833527 seconds
That is 0.22363211884700432 seconds per sample
Train shape: (7560, 80, 64, 1). Test shape: (2160, 80, 64, 1)
Classification accuracy for ./model/EEGNet_fusion_Executed_103sub_0.h5 : 0.943056 


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2024-12-14 02:58:37.682641: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
Traceback (most recent call last):
  File "run_experiments.py", line 93, in <module>
    experiments.append(run_experiment(X, y, experiment_103sub))
  File "d:\CodesNStuffs\NeuralScience\BCI_mini_project\EEGMotorImagery\training_testing.py", line 131, in run_experiment
    multi_branch, experiment.get_epochs(), test_model=test_model)
  File "d:\CodesNStuffs\NeuralScience\BCI_mini_project\EEGMotorImagery\training_testing.py", line 68, in train_test_model
    acc, equals, preds = predict_accuracy(model, X_test, y_test, new_model_name, multi_branch=multi_branch)
ValueError: not enough values to unpack (expected 3, got 2)


# Load model

In [2]:
from tensorflow.keras.models import load_model

# Step 1: Load the trained model
model_path = './model/EEGNet_fusion_Executed_103sub_0.h5'
model = load_model(model_path)

# Optional: Print model summary to ensure it's loaded correctly
model.summary()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 80, 64, 1)]  0      

# Testing 

In [3]:
from data_loader import load_data
from run_type import RunType
from tensorflow.keras.utils import to_categorical

# Step 2: Load and preprocess the test data
nr_of_subj = 50  # Number of subjects to test
trial_type = RunType.Executed  # Or RunType.Imagined based on your data
use_cpu = False  # Use CPU or GPU depending on your setup

# Load test data (with preprocessing)
X_test, y_test = load_data(nr_of_subj=nr_of_subj, trial_type=trial_type, chunk_data=True, chunks=8,
                           cpu_format=use_cpu, preprocessing=True, hp_freq=0.5, bp_low=2, bp_high=60, notch=True,
                           hp_filter=False, bp_filter=True, artifact_removal=True)

# Format the labels to categorical
y_test = to_categorical(y_test, num_classes=2)  # Update num_classes as per your case


Loaded data shapes:
(17640, 64, 80) (17640, 1)


In [4]:
# Step 3: Reshape data for the model (add channel dimension)
# The model expects input of shape (80, 64, 1) but your data is in (64, 80, 1)
# So we need to transpose the data dimensions

X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)  # Shape: (samples, height, width, 1)

# Transpose data from (64, 80, 1) to (80, 64, 1)
X_test_reshaped = X_test_reshaped.transpose(0, 2, 1, 3)  # Transpose the last two dimensions

# Prepare the data as 3 copies (multi-branch input)
X_test_input = [X_test_reshaped, X_test_reshaped, X_test_reshaped]


In [5]:
# Step 4: Make predictions using the trained model
predictions = model.predict(X_test_input)

# Convert predictions to class labels (0 or 1)
predicted_classes = predictions.argmax(axis=-1)

# Convert the ground truth labels from one-hot encoding to integers
y_test_classes = y_test.argmax(axis=-1)


In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Step 5: Evaluate the model's performance
# Calculate accuracy
accuracy = accuracy_score(y_test_classes, predicted_classes)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Confusion matrix
cm = confusion_matrix(y_test_classes, predicted_classes)
print("Confusion Matrix:\n", cm)


Test accuracy: 50.41%
Confusion Matrix:
 [[4767 4105]
 [4643 4125]]


In [7]:
import pandas as pd

results = {
    'True Labels': y_test_classes,
    'Predicted Labels': predicted_classes
}

# Create a DataFrame
df = pd.DataFrame(results)

# Save to CSV
csv_file = 'predicted_vs_true_labels.csv'
df.to_csv(csv_file, index=False)